# Erste Schritte mit Hugging Face
<br>
<a href="https://huggingface.co"><img src="../img/huggingface_small.png" width="30%">
<br>https://huggingface.co</a>
<br>



# Textklassifikation (Text Classification)

Sprachmodelle sind in der Lage, Texte zu **klassifizieren**.

Das bedeutet, dass das Ergebnis der Verarbeitung eines Textes die Aussage wie wahrscheinlich es ist, dass ein Text _positiv_ oder _negativ_ konnotiert ist. Diese Ausgabe ist also eine prozentuale Klassenzugehörigkeit zu den Klassen "POSITIVE" oder "NEGATIVE".


---
Beispiele angelehnt an

Tunstall, Lewis; Werra, Leandro von; Wolf, Thomas (2023): Natural Language Processing mit Transformern. Sprachanwendungen mit Hugging Face erstellen. Heidelberg: O'Reilly (Animals). Online verfügbar unter https://nbn-resolving.org/urn:nbn:de:bsz:31-epflicht-2107005.     


In [1]:
text = """Dear Amazon, last week I ordered an Optimus Prime ac- tion figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.""" 
# vgl. Tunstall et al. 2023, S. 10

In [2]:
from transformers import pipeline
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")


In [3]:
import pandas as pd
outputs = classifier(text) 
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.87243


# Erkennung von Namens-Entitäten (Named Entity Recognition)

Sprachmodelle sind in der Lage, Produkte, Orte oder Eigennamen aus Texten zu extrahieren. Diese werden als "Named Entity" bezeichnet.

In [4]:
ner_tagger = pipeline("ner", aggregation_strategy="simple", model="dbmdz/bert-large-cased-finetuned-conll03-english")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,entity_group,score,word,start,end
0,ORG,0.855356,Amazon,5,11
1,MISC,0.992322,Optimus Prime,36,49
2,LOC,0.999735,Germany,92,99
3,MISC,0.606780,Mega,210,214
4,PER,0.524000,##tron,214,218
5,ORG,0.630277,Decept,255,261
6,MISC,0.506672,##icons,261,266
7,MISC,0.772444,Megatron,352,360
8,MISC,0.988840,Optimus Prime,369,382
9,PER,0.814836,Bumblebee,504,513


Auch hier ist das Ergebnis eine statistische Zuordnung von Wörtern zu Kategorien mit der Angabe von Wahrscheinlichkeiten, die die so genannte "confidence" des Modells angeben. Dies ist die "Sicherheit" mit der das Modell die Zuordnung einschätzt. Kategorien sind z.B. "Organisation (ORG)", "Location (LOC)" oder "Person (PER)".

Zusätzlich gibt das Modell einen Bereich an, in dem es das Wort (oder die Wortgruppe) gefunden hat.

In der Ausgabe können sich auch noch Tokens (mit ## gekennzeichnet) befinden, wenn kein ganzes Wort oder eine Wortgruppe einer Kategorie zugeordnet werden konnte. Interessant ist hier, dass letztlich das "Plural-Token" separat behandelt wurde (##ns stammt aus dem Wort "Decepticons").

# Beantwortung von Fragen (Question Answering)

Sprachmodelle können Fragen beantworten. Dies setzt natürlich voraus, dass das Modell einen Kontext kennt zu dem Fragen beantwortet werden sollen.
Hier wollen wir eine Frage zum obigen Text beantworten, der dem Modell bereits bekannt ist. Wir wollen folgende Frage stellen:

**What does the customer want?**

In [5]:
reader = pipeline("question-answering", model="distilbert/distilbert-base-cased-distilled-squad")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

,score,start,end,answer
0,0.533085,337,360,an exchange of Megatron


Auch hier wird zusätzlich zur Antwort ein Score ausgegeben, der die confidence des Modells angibt.
<blockquote>Beachten Sie: 

Es handelt sich hier nicht um einen generativen Ansatz, sondern letztlich auch um eine Text-Klassifizierung. Das Ergebnis des Modells ist eine Textstelle, in der das Modell die Antwort vermutet. Es handelt sich bei diesem Ansatz um so genanntes **"Extractive Question Answering"**. Hierfür gibt das Modell die Spann mit "start" und "end" aus, in dem es die Antwort vermutet.</blockquote>


# Textzusammenfassung (Summarization)

Sprachmodelle sind in der Lage, Texte zusammenzufassen. Hierfür gibt es verschiedene methodische Ansätze. Zunäschst wollen wir nur ein einführendes Beispiel betrachten.

In [6]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summary = summarizer(text, max_length=150, min_length=40, do_sample=False)

Your max_length is set to 150, but your input_length is only 113. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)


In [7]:
summary[0]['summary_text']

'Bumblebee ordered an Optimus Prime figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I hope you can understand my dilemma.'

Auffällig sind am Ergebnis zwei Punkte:
- Das Modell hat erkannt, dass der Autor "Bumblebee" ist.
- Ansonsten sind im Ergebnis letztlich Textbestandteile zusammenkopiert.
- Der eigentliche Wunsch auf "Austausch" der Figur wurde nicht in der Zusammenfassung erwähnt.

Weiterführende Anleitung bei Keras: https://keras.io/examples/nlp/t5_hf_summarization/

# Übersetzen
[Credits to: https://huggingface.co/docs/transformers/task_summary]

In [8]:
from transformers import pipeline

text = "translate English to French: Hugging Face is a community-based open-source platform for machine learning."
translator = pipeline(task="translation", model="google-t5/t5-small", max_length=400)


C:\Users\thomase\miniconda3\envs\studientage-pytorch\lib\site-packages\transformers\pipelines\__init__.py:1086: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


In [10]:
translator(text)

[{'translation_text': "Hugging Face est une tribune communautaire de l'apprentissage des machines."}]

# Text-Generierung

In [11]:
from transformers import pipeline

prompt = "What is machine learning?"
generator = pipeline(task="text-generation", model="openai-community/gpt2")
result = generator(prompt)  # doctest: +SKIP

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [12]:
result[0]['generated_text']

'What is machine learning?\n\nMachine learning (or ML, to be more precise) is a statistical programming language which uses code analysis and algorithms to identify tasks, predict outcomes or learn how to perform them. It has always been a field that has'

# Fazit

Wir konnten sehen, dass bereits mit dem direkten Einsatz vortrainierter Modell größtenteils brauchbare Ergebnisse erzielt werden können.
Um nun tatsächlich gute Ergebnisse zu erzielen werden wir tiefer eintauchen.  